In [6]:
import Pkg; Pkg.add("Combinatorics")
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, Statistics, Random, Distributions, DataFrames, Combinatorics
using Dates

    Updating registry at `C:\Users\Arushi Jain\.julia\registries\General.toml`
   Resolving package versions...
   Installed Combinatorics ─ v1.0.2
    Updating `C:\Users\Arushi Jain\.julia\environments\v1.8\Project.toml`
  [861a8166] + Combinatorics v1.0.2
    Updating `C:\Users\Arushi Jain\.julia\environments\v1.8\Manifest.toml`
  [861a8166] + Combinatorics v1.0.2


In [7]:
function generateDistribution(n, p)
    m = floor(Int, n*p)
    a = ones(Int, m)
    b = zeros(Int, n-m)
    res = [a;b]
    return shuffle(res)
end

generateDistribution (generic function with 1 method)

In [10]:
t1 = now()

2022-11-29T17:47:18.767

In [11]:
toc = now()

2022-11-29T17:47:27.807

In [17]:
time_difference = Dates.DateTime(toc) - Dates.DateTime(t1)

9040 milliseconds

In [21]:
time_difference < Minute(30)

true

In [46]:
function brute_force(s,t,A,b)
    n = length(s)    
    tic = now()
    # Generate new s
    for i in 0:n
        s_f = vcat(repeat([1],i), repeat([0],n-i))
        for s_try in multiset_permutations(s_f,length(s_f)) 
            if sum((A*s_try+b).%2) <= t
                return s_try
            end

            toc = now()
            if toc-tic > Minute(30)
                return s_try
            end
        end 
    end 

end 



brute_force (generic function with 1 method)

In [1]:
function MIO(A,b,n,m)
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)
    set_optimizer_attribute(model, "TimeLimit", 1800)  # limit 30 minutes 
    epsilon = 0.0001

    # Add variables
    @variable(model, s[i=1:n], Bin)
    @variable(model, h[j = 1:m] >= 0)
    @variable(model, z[j = 1:m] >= 0)
    @variable(model, y[j = 1:m] >= 0, Int)

    @objective(model, Min, sum(z[j] for j = 1:m))

    @constraint(model, [j=1:m] ,   h[j] - b[j]  <= z[j])
    @constraint(model, [j=1:m] , - h[j] + b[j]  <= z[j])
    @constraint(model, [j=1:m] , sum(A[j,i]*s[i] for i = 1:n)  == 2*y[j] + h[j])
    @constraint(model, [j=1:m] ,  h[j] <= 2 - epsilon)

    time1 = @elapsed optimize!(model)
    return value.(s), time1
end 

LoadError: LoadError: UndefVarError: @variable not defined
in expression starting at In[1]:8

## MIO

In [10]:
seed = 15095
p = 0.1
results = zeros(5000, 6)
num_result = 1
for n=[4:2:20;]
#for n=[4:2:4;]
    test = Matrix(I, n, n);
    M = floor(Int,4*n*((0.5-p)^(-2)) )
    for ratio in [0, 0.25, 0.5, 0.75, 1]
        s = generateDistribution(n, ratio)

        for m = [Int(trunc(M/50)):Int(trunc(M/50)):Int(trunc(M/10));] 
            count = 0
            threshold = ceil(Int, m*p + sqrt(n*m))
            A = bitrand((m,n))
            #e = generateDistribution(m, p)
            #d = Binomial(1,p)
            e = rand(Binomial(1,p), m)
            b = (A*s + e).%2;

            # Solve MIO
            s_pred, time1 =  MIO(A,b,n,m);

            # Verify Correctness
            error = sum((A*s_pred+b).%2)
            if error < threshold
                count += 1
            end
        
            # Print Result
            println("For n=$n, ratio=$ratio, threshold=$threshold, key $s, m=$m,  error=$error")          
            result = [n, m, threshold, ratio, count, time1] # count 1 recovered
            results[num_result, :] = result
            num_result += 1
        end
    end
end

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=4, ratio=0.0, threshold=8, key [0, 0, 0, 0], m=10,  error=2.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=4, ratio=0.0, threshold=11, key [0, 0, 0, 0], m=20,  error=1.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=4, ratio=0.0, threshold=14, key [0, 0, 0, 0], m=30,  error=2.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=4, ratio=0.0, threshold=17, key [0, 0, 0, 0], m=40,  error=3.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=4, ratio=0.0, threshold=20, key [0, 0, 0, 0], m=50,  error=4.0
Set parameter Usernam

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=6, ratio=0.0, threshold=36, key [0, 0, 0, 0, 0, 0], m=105,  error=17.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=6, ratio=0.0, threshold=39, key [0, 0, 0, 0, 0, 0], m=120,  error=15.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=6, ratio=0.0, threshold=42, key [0, 0, 0, 0, 0, 0], m=135,  error=19.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=6, ratio=0.0, threshold=45, key [0, 0, 0, 0, 0, 0], m=150,  error=14.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=6, ratio=0.125, threshold=11, key [0, 0, 0, 0, 0, 0

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=8, ratio=0.0, threshold=34, key [0, 0, 0, 0, 0, 0, 0, 0], m=80,  error=7.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=8, ratio=0.0, threshold=39, key [0, 0, 0, 0, 0, 0, 0, 0], m=100,  error=6.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=8, ratio=0.0, threshold=43, key [0, 0, 0, 0, 0, 0, 0, 0], m=120,  error=10.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=8, ratio=0.0, threshold=48, key [0, 0, 0, 0, 0, 0, 0, 0], m=140,  error=20.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=8, ratio=0.0, threshold=52, ke

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=8, ratio=0.5, threshold=34, key [1, 0, 1, 1, 0, 0, 0, 1], m=80,  error=10.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=8, ratio=0.5, threshold=39, key [1, 0, 1, 1, 0, 0, 0, 1], m=100,  error=10.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=8, ratio=0.5, threshold=43, key [1, 0, 1, 1, 0, 0, 0, 1], m=120,  error=10.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=8, ratio=0.5, threshold=48, key [1, 0, 1, 1, 0, 0, 0, 1], m=140,  error=16.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=8, ratio=0.5, threshold=52, 

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=10, ratio=0.375, threshold=28, key [1, 0, 0, 1, 0, 0, 0, 0, 0, 1], m=50,  error=6.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=10, ratio=0.375, threshold=35, key [1, 0, 0, 1, 0, 0, 0, 0, 0, 1], m=75,  error=10.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=10, ratio=0.375, threshold=42, key [1, 0, 0, 1, 0, 0, 0, 0, 0, 1], m=100,  error=13.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=10, ratio=0.375, threshold=48, key [1, 0, 0, 1, 0, 0, 0, 0, 0, 1], m=125,  error=15.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60


...

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=28, ratio=0.25, threshold=98, key [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1], m=210,  error=81.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=28, ratio=0.25, threshold=117, key [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1], m=280,  error=112.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=28, ratio=0.25, threshold=134, key [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1], m=350,  error=145.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=28, ratio=0.25, threshold=151, key [1, 0, 0, 0,

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=30, ratio=0.0, threshold=105, key [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], m=225,  error=24.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=30, ratio=0.0, threshold=125, key [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], m=300,  error=29.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=30, ratio=0.0, threshold=144, key [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], m=375,  error=38.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=30, ratio=0.0, threshold=162, key

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=30, ratio=0.375, threshold=105, key [0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0], m=225,  error=81.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=30, ratio=0.375, threshold=125, key [0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0], m=300,  error=122.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=30, ratio=0.375, threshold=144, key [0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0], m=375,  error=28.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=30, ratio=0.375, threshold

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=32, ratio=0.125, threshold=112, key [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], m=240,  error=85.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=32, ratio=0.125, threshold=134, key [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], m=320,  error=130.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=32, ratio=0.125, threshold=154, key [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], m=400,  error=164.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=32, rat

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=32, ratio=0.5, threshold=88, key [0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1], m=160,  error=55.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=32, ratio=0.5, threshold=112, key [0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1], m=240,  error=90.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=32, ratio=0.5, threshold=134, key [0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1], m=320,  error=131.0
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 60
For n=32, ratio=0.5, 

In [11]:
df = DataFrame(results, :auto)
df = rename!(df,[:n,:m,:threshold,:ratio,:count,:time])
CSV.write("results_MIO_till32.csv",df)

"results_MIO_till32.csv"

In [43]:
df

Row,n,m,threshold,ratio,count,time
,Float64,Float64,Float64,Float64,Float64,Float64
1,4.0,20.0,11.0,0.0,1.0,0.000418333
2,4.0,40.0,17.0,0.0,1.0,0.000973625
3,4.0,60.0,22.0,0.0,1.0,0.00134154
4,4.0,80.0,26.0,0.0,1.0,0.00232633
5,4.0,100.0,30.0,0.0,1.0,0.00216729
6,4.0,20.0,11.0,0.125,1.0,0.0005665
7,4.0,40.0,17.0,0.125,1.0,0.000908291
8,4.0,60.0,22.0,0.125,1.0,0.00120838
9,4.0,80.0,26.0,0.125,1.0,0.00151979


# Brute force

In [47]:
seed = 15095
p = 0.1
results = zeros(5000, 6)
num_result = 1
for n=[4:2:20;]
#for n=[4:2:4;]
    test = Matrix(I, n, n);
    M = floor(Int,4*n*((0.5-p)^(-2)) )
    for ratio in [0, 0.25, 0.5, 0.75, 1]
        s = generateDistribution(n, ratio)

        for m = [Int(trunc(M/50)):Int(trunc(M/50)):Int(trunc(M/10));] 
            count = 0
            threshold = ceil(Int, m*p)
            A = bitrand((m,n))
            e = generateDistribution(m, p)
            b = (A*s + e).%2;

            # Solve Brute force
            s_pred = brute_force(s,threshold,A,b);
            time1 =  @elapsed brute_force(s,threshold,A,b);

            # Verify Correctness
            error = sum((A*s_pred+b).%2)
            if error <= threshold
                count += 1
            end
            recover = (s_pred == s)
            # Print Result
            println("For n=$n, ratio=$ratio, threshold=$threshold, key $s, m=$m, recover=$recover, error=$error")          
            result = [n, m, threshold, ratio, count, time1] # count 1 recovered
            results[num_result, :] = result
            num_result += 1
        end
    end
end

For n=4, ratio=0.0, threshold=1, key [0, 0, 0, 0], m=2, recover=true, error=0
For n=4, ratio=0.0, threshold=1, key [0, 0, 0, 0], m=4, recover=true, error=0
For n=4, ratio=0.0, threshold=1, key [0, 0, 0, 0], m=6, recover=true, error=0
For n=4, ratio=0.0, threshold=1, key [0, 0, 0, 0], m=8, recover=true, error=0
For n=4, ratio=0.0, threshold=1, key [0, 0, 0, 0], m=10, recover=true, error=1
For n=4, ratio=0.25, threshold=1, key [1, 0, 0, 0], m=2, recover=false, error=1
For n=4, ratio=0.25, threshold=1, key [1, 0, 0, 0], m=4, recover=false, error=1
For n=4, ratio=0.25, threshold=1, key [1, 0, 0, 0], m=6, recover=true, error=0
For n=4, ratio=0.25, threshold=1, key [1, 0, 0, 0], m=8, recover=true, error=0
For n=4, ratio=0.25, threshold=1, key [1, 0, 0, 0], m=10, recover=true, error=1
For n=4, ratio=0.5, threshold=1, key [1, 1, 0, 0], m=2, recover=false, error=1
For n=4, ratio=0.5, threshold=1, key [1, 1, 0, 0], m=4, recover=false, error=1
For n=4, ratio=0.5, threshold=1, key [1, 1, 0, 0], m=

For n=12, ratio=0.5, threshold=2, key [0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0], m=18, recover=false, error=2
For n=12, ratio=0.5, threshold=3, key [0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0], m=24, recover=false, error=3
For n=12, ratio=0.5, threshold=3, key [0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0], m=30, recover=true, error=3
For n=12, ratio=0.75, threshold=1, key [0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1], m=6, recover=false, error=1
For n=12, ratio=0.75, threshold=2, key [0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1], m=12, recover=false, error=2
For n=12, ratio=0.75, threshold=2, key [0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1], m=18, recover=false, error=2
For n=12, ratio=0.75, threshold=3, key [0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1], m=24, recover=false, error=3
For n=12, ratio=0.75, threshold=3, key [0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1], m=30, recover=true, error=3
For n=12, ratio=1.0, threshold=1, key [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], m=6, recover=false, error=1
For n=12, ratio=1.0, threshold=2, key [1, 1, 1, 1, 1, 

For n=18, ratio=0.5, threshold=2, key [1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1], m=18, recover=false, error=2
For n=18, ratio=0.5, threshold=3, key [1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1], m=27, recover=false, error=3
For n=18, ratio=0.5, threshold=4, key [1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1], m=36, recover=true, error=3
For n=18, ratio=0.5, threshold=5, key [1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1], m=45, recover=true, error=4
For n=18, ratio=0.75, threshold=1, key [1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1], m=9, recover=false, error=1
For n=18, ratio=0.75, threshold=2, key [1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1], m=18, recover=false, error=2
For n=18, ratio=0.75, threshold=3, key [1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1], m=27, recover=false, error=1
For n=18, ratio=0.75, threshold=4, key [1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1], m=36, recover=false, error=4
For n=18, ratio

In [ ]:
df = DataFrame(results, :auto)
df = rename!(df,[:n,:m,:threshold,:ratio,:count,:time])
CSV.write("results_bruteforce_till32.csv",df)


In [17]:
df 

Row,x1,x2,x3,x4
,Float64,Float64,Float64,Float64
1,4.0,34.0,0.0,25.0
2,4.0,34.0,0.125,25.0
3,4.0,34.0,0.25,25.0
4,4.0,34.0,0.375,25.0
5,4.0,34.0,0.5,25.0
6,6.0,57.0,0.0,25.0
7,6.0,57.0,0.125,25.0
8,6.0,57.0,0.25,25.0
9,6.0,57.0,0.375,25.0
